In [6]:
import requests
import time
import random
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

In [7]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 'cu=es-es; reese84=3:+1TDsoHwKvg7f1wOMN9tvg==:z3WnC+XP/wZGCyqs7t+HUGqdBhezC47trmfQny+9hR5B3ADv6Xw6As90OTgqf6Rs3su3rjCkaub1cEETzSFvOh8hYrNd1Suw2CvW8DaXBYvUkoKI27hHgGMzUlX7TzK9IMaMouwAtqn1jKWsCNMJXG7SyNWH7/zXikWwT1OUX6P0bJiycu3iczVx0AIxLf0y7aCrdWp6WQsvmEdBVjC2e21uQRk1SzpUH8cCoPFwDplXxY7tbskeRPpZs8Sqq9zAQqOQ9OCseQk+m3jnEA+V3v9JA/2/AzjqTyWT5xAFmT16srmrxo1UXev7zM0ORTECZdDU9nPagz9LHMeywCL1Cu05szHKHu31AUFTD9djMqvH2WRuHBT+BfJ/K52ftqVYZB1GC2EVp4BoEasVlWNAqXe08QogndLr4lLXOWlt8zUsqgqd/AU5DxXhU/7Uxq9wo5GC3/Nv9MbdSbr9xZDPzA==:rytuKWTipIT1AmbcGpQ5bqEakjhbToxv43vaTrNDkIA=; re_uuid=e89eb6fb-e995-49ff-8215-770ac8882b22; AMCVS_05FF6243578784B37F000101%40AdobeOrg=1; AMCV_05FF6243578784B37F000101%40AdobeOrg=-408604571%7CMCIDTS%7C20119%7CMCMID%7C37440691961961117764348177306153016152%7CMCAAMLH-1738867687%7C6%7CMCAAMB-1738867687%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1738270087s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.6.0; ab.storage.deviceId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3Afc8d286a-0bd3-9fa7-77d9-c101af31aba6%7Ce%3Aundefined%7Cc%3A1738262887448%7Cl%3A1738262887448; ajs_anonymous_id=2983f013-a5c7-46af-98d2-45e6775450de; _fbp=fb.1.1738262887799.564221553494210634; didomi_token=eyJ1c2VyX2lkIjoiMTk0Yjg4YWItYjM2OS02ODg0LTk2NWQtNzdkODZiNTg2OGRlIiwiY3JlYXRlZCI6IjIwMjUtMDEtMzBUMTg6NDg6MDcuMjIyWiIsInVwZGF0ZWQiOiIyMDI1LTAxLTMwVDE4OjQ4OjEwLjA1NVoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmFkZXZpbnRhLW1vdG9yLW1vdG9yIiwiYzptZXRhcGxhdGYtTlJlVnBtTFEiLCJjOmNyaXRlby1QNDhlR1QydyIsImM6Z29vZ2xlaXJlLWZmS2FQYVJEIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbImRldmljZV9jaGFyYWN0ZXJpc3RpY3MiLCJnZW9sb2NhdGlvbl9kYXRhIiwidHJhbnNmZXItdG8tbW90b3IiLCJ1c29kZWxvcy1RcExwTThqVyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; euconsent-v2=CQMC4oAQMC4oAAHABBENBaFsAP_gAEPgAAiQKxtX_G__bWlr8X73aftkeY1P99h77sQxBhbJE-4FzLvW_JwXx2E5NA36tqIKmRIAu3TBIQNlHJDURVCgaogVrSDMaEyUoTNKJ6BkiFMRI2dYCFxvm4tjeQCY5vr991dx2B-t7dr83dzyy4hHn3a5_2S0WJCdA5-tDfv9bROb-9IOd_x8v4v4_F7pE2_eT1l_tWvp7D9-cts_9XW99_fbff9Pn_-uB_-_X_vf_H37gq-ASYaFRAGWBISEGgYQQIAVBWEBFAgCAABIGiAgBMGBTsDABdYSIAQAoABggBAACDIAEAAAEACEQAQAFAgAAgECgADAAgGAgAIGAAEAFgIBAACA6BimBBAIFgAkZkVCmBCEAkEBLZUIJAECCuEIRZ4BEAiJgoAAAAACkAAQFgsDiSQEqEggC4gmgAAIAEAggAKEEnJgACAM2WoPBk2jK0wDB8wSIaYBkARBGQkGgAAA.f_wACHwAAAAA; _gcl_au=1.1.1672240763.1738262890; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7IjEiOnRydWUsIjIiOnRydWUsIjMiOnRydWUsIjQiOnRydWUsIjUiOnRydWUsIjYiOnRydWUsIjciOnRydWUsIjgiOnRydWUsIjkiOnRydWUsIjEwIjp0cnVlfX0sInNwZWNpYWxGZWF0dXJlcyI6eyIxIjp0cnVlfX0=; segment_ga=GA1.1.1243879313.1738262890; kanirante-adv-aauid=4985cf5d-b463-4c02-9c52-2d7ad64ff03a; adit-xandr-id=8784207916437135491; __gads=ID=59effd87b1f14b99:T=1738262892:RT=1738262892:S=ALNI_MYeyJZVUcWpxXnUxpM9MdNKua67Yg; __gpi=UID=00000ff50f2c4710:T=1738262892:RT=1738262892:S=ALNI_MZcC92RKXmkbbRl9umNH2XmdCv-eg; __eoi=ID=3c0856d3cf4f047c:T=1738262892:RT=1738262892:S=AA-AfjZxOC_13DTICpZTXGxg3guK; _hjSessionUser_89611=eyJpZCI6IjQ0YWI5NDFiLTAyOTItNTgxNy04YzFjLTU0M2MzZDIwYWRiNSIsImNyZWF0ZWQiOjE3MzgyNjI4OTc2NDksImV4aXN0aW5nIjpmYWxzZX0=; _hjSession_89611=eyJpZCI6ImQxM2QxOGEyLTllY2YtNDdiNy1hMDliLWU1ZjMwNzk2MGM1NiIsImMiOjE3MzgyNjI4OTc2NTAsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MH0=; sui_1pc=17382628994231538712CEE53BE0A0D26B183F7D8B419009A9DE7FC8; _lr_retry_request=true; _lr_env_src_ats=false; __gsas=ID=c5c2e8ef834d9dfa:T=1738262905:RT=1738262905:S=ALNI_MafdAaQ41sBEkyyFHXBAqMBA5o4Tg; _csrf=DPlwpDaYiKqFxSzOYGVHiBepeIBwg7YIbizvB0SrQgXbhfoNfK3FR/Z1iIwAH5eMw10kAFu1xOP2c5ZcLa+iKLeR81WitLZ+TbJz+qaD7bg=; _lr_sampling_rate=100; ShowMap=false; segment_ga_MH0GDRSFGC=GS1.1.1738262890.1.1.1738262950.60.0.0; ab.storage.sessionId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3Adb6deb3f-d1b7-bc76-ed18-f1cf24940a30%7Ce%3A1738264750618%7Cc%3A1738262887447%7Cl%3A1738262950618; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22DHaIOswqasf637vjK8Xs%22%2C%22expiryDate%22%3A%222026-01-30T18%3A49%3A10.685Z%22%7D; cto_bundle=TqDhHl91bFFLU3lhZFFocnp3TjNrbkludDRLeUxDWFBIYVFlMTlnQ2ZhdkdnQzRPUHc1U3JCRTVaR0l2bCUyQjJPNXZVRVExVko5MkhhdElPRWRRdUc5NmJYY05IJTJGT2kweGhraFR4YUNDRU5iUjFqWndiYTJLNkwyYWpTVVlzejlieVVidEI; __rtbh.uid=%7B%22eventType%22%3A%22uid%22%2C%22id%22%3A%22unknown%22%2C%22expiryDate%22%3A%222026-01-30T18%3A49%3A11.761Z%22%7D',
    "Referer": "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/l",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "X-Height": "9999",  # Establece el valor de height
    "X-Width": "1920",   # Puedes agregar otros parámetros, como width
    "X-Depth": "500",    # Ejemplo de parámetro depth, si es necesario
    "X-Color": "blue",   # Ejemplo de parámetro de color, si lo deseas
    "X-Orientation": "landscape"  # Otro ejemplo, si necesitas un parámetro de orientación
}


In [9]:
# Crear una sesión
def check_error_page(html_page):
    # Patrón para buscar frases comunes en una página de error
    patrones_error = [
        r"sentimos la interrupciã\x93n",  # Detecta el mensaje de "Sentimos la interrupción"
    ]
    
    # Convertir el HTML a minúsculas para hacerlo insensible a mayúsculas
    html_lower = html_page.lower()
    # Buscar cada uno de los patrones de error en el HTML
    for patron in patrones_error:
        if re.search(patron, html_lower):
            return True
    return False
    
# Función para obtener los detalles del inmueble
def extract_property_info(property_data):
    # Titulo del inmueble: Se puede obtener desde la dirección del detalle
    titulo = f"{property_data['address']['city']} - {property_data['address']['district']}"

    # Precio (lo extraemos del campo 'price')
    precio = property_data['price']

    # Habitaciones: 'rooms' en las características
    habitaciones = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'rooms'), None)

    # Convertir a int si no es None
    habitaciones = int(habitaciones) if habitaciones is not None else 0

    # Baños: 'bathrooms' en las características
    banos = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'bathrooms'), None)

    # Superficie: 'surface' en las características
    superficie = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'surface'), None)

    # Convertir a int si no es None
    superficie = int(superficie) if superficie is not None else 0

    # Planta: 'floor' en las características
    planta = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'floor'), None)

    # Convertir a int si no es None
    planta = int(planta) if planta is not None else 0

    # Enlace: detalle completo
    full_link = property_data['detail'].get('es-ES', '')

    return {
        "titulo": titulo,
        "precio": precio,
        "habitaciones": habitaciones,
        "banos": banos,
        "superficie": superficie,
        "planta": planta,
        "enlace": full_link,
        'fuente': 'fotocasa',
    }

session = requests.Session()
session.headers.update(headers)

# URL base
base_url = "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/"

# Función para verificar si es una página de error
def check_error_page(html_page):
    patrones_error = [r"sentimos la interrupciã\x93n"]
    html_lower = html_page.lower()
    for patron in patrones_error:
        if re.search(patron, html_lower):
            return True
    return False

# Función para extraer los datos de un inmueble según JSON
def extract_property_info(property_data):
    try:
        titulo = f"{property_data['address']['city']} - {property_data['address']['district']}"
        precio = property_data.get('price', None)
        habitaciones = int(next((f['value'] for f in property_data['features'] if f['key'] == 'rooms'), 0))
        banos = int(next((f['value'] for f in property_data['features'] if f['key'] == 'bathrooms'), 0))
        superficie = int(next((f['value'] for f in property_data['features'] if f['key'] == 'surface'), 0))
        planta = int(next((f['value'] for f in property_data['features'] if f['key'] == 'floor'), 0))
        full_link = property_data['detail'].get('es-ES', '')
        return {
            "titulo": titulo,
            "precio": precio,
            "habitaciones": habitaciones,
            "banos": banos,
            "superficie": superficie,
            "planta": planta,
            "enlace": full_link,
            "fuente": "fotocasa"
        }
    except Exception as e:
        print(f"Error extrayendo datos del inmueble: {e}")
        return None

# Función para manejar diferentes selectores según la estructura de la página
def extract_data_with_selectors(html, json_data=None):
    inmuebles = []

    # Intentar extraer desde JSON
    if json_data:
        try:
            real_estates = json_data['initialSearch']['result']['realEstates']
            for inmueble in real_estates:
                info = extract_property_info(inmueble)
                if info:
                    inmuebles.append(info)
        except KeyError:
            print("El JSON no contiene la clave esperada para 'realEstates'.")

    # Intentar extraer desde HTML usando BeautifulSoup (alternativa)
    try:
        soup = BeautifulSoup(html, 'html.parser')
        cards = soup.find_all('article', class_='re-CardPackPremium')
        for card in cards:
            titulo = card.find('h3', class_='re-CardHeader')
            precio = card.find('span', class_='re-CardPrice')
            enlace = card.find('a', class_='re-CardPackPremium-info-container')
            if titulo and precio and enlace:
                inmuebles.append({
                    "titulo": titulo.get_text(strip=True),
                    "precio": precio.get_text(strip=True),
                    "habitaciones": "N/A",
                    "banos": "N/A",
                    "superficie": "N/A",
                    "planta": "N/A",
                    "enlace": f"https://www.fotocasa.es{enlace['href']}",
                    "fuente": "fotocasa"
                })
    except Exception as e:
        print(f"Error extrayendo datos con BeautifulSoup: {e}")

    return inmuebles

# Paginación y scraping
page = 1
all_listings = []
page_limit = 30

while page < page_limit:
    try:
        print(f"Procesando página {page}...")
        response = session.get(f"{base_url}l/{page}")
        response.raise_for_status()
        error_page = check_error_page(response.text)
        if error_page:
            print("Bloqueado por Fotocasa, cambie la cookie")
            break

        # Intentar extraer JSON
        json_data = None
        match = re.search(
            r'window\.__INITIAL_PROPS__ = JSON\.parse\("((?:[^"\\]|\\.)*)"\);',
            response.text
        )
        if match:
            escaped_json = match.group(1)
            try:
                json_data = json.loads(escaped_json.encode('utf-8').decode('unicode_escape'))
            except json.JSONDecodeError:
                print("Error al decodificar el JSON")

        # Extraer datos usando múltiples estrategias
        inmuebles = extract_data_with_selectors(response.text, json_data)
        all_listings.extend(inmuebles)

        # Pasar a la siguiente página
        page += 1
        time.sleep(random.uniform(5, 10))  # Espera aleatoria para evitar bloqueos

    except Exception as e:
        print(f"Error procesando la página {page}: {e}")
        break

# Convertir a DataFrame
df = pd.DataFrame(all_listings)
df

Procesando página 1...
Bloqueado por Fotocasa, cambie la cookie


""


In [9]:
df

,titulo,precio,habitaciones,banos,superficie,planta,enlace,fuente
0,Murcia Capital - Norte,580 €,1,1,49,4,/es/alquiler/vivienda/murcia-capital/ascensor-...,fotocasa
1,La Manga del Mar Menor - Entremares,650 €,2,1,51,7,/es/alquiler/vivienda/la-manga-del-mar-menor/a...,fotocasa
2,Cartagena - Cartagena ciudad,800 €,2,1,87,9,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
3,Murcia Capital - Pedanías Este,695 €,2,1,75,6,/es/alquiler/vivienda/murcia-capital/aire-acon...,fotocasa
4,Murcia Capital - Centro,680 €,2,1,63,0,/es/alquiler/vivienda/murcia-capital/san-andre...,fotocasa
...,...,...,...,...,...,...,...,...
145,Murcia Capital - Pedanías Oeste,790 €,3,2,100,0,/es/alquiler/vivienda/murcia-capital/terraza-a...,fotocasa
146,Mazarrón - Puerto de Mazarrón,2.950 €,3,2,280,0,/es/alquiler/vivienda/mazarron/aire-acondicion...,fotocasa
147,Cartagena - Cartagena ciudad,1.200 €,3,2,120,0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
148,Cartagena - Cartagena ciudad,900 €,3,1,83,0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa


In [10]:
#df.to_csv('alquileres_fotocasa.csv')